<a href="https://colab.research.google.com/github/shreyasmiraj/lncRNA_Analysis/blob/main/rna_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation

In [ ]:
!pip install CFsshTunnel
!pip install biopython

#Imports

In [ ]:
import os
import Bio
import pickle
import numpy as np
import pandas as pd

from getpass import getpass
from google.colab import userdata
from CFsshTunnel.CFsshTunnel import CFsshTunnel
from CFsshTunnel.utils.utils import keep_alive
from CFsshTunnel.code_server.code_server import launch_codeserver

#CFsshTunnel


In [ ]:
_, hostname, user = CFsshTunnel(public_keys=[userdata.get('drsa')])

Checking for openssh-server
openssh-server already installed
Checking for cloudflared
cloudflared already installed


+---------------------------------------------------+
| open_ssh_cloudflare tunnel route is now alive at: |
| ft-screening-underground-wave.trycloudflare.com   |
+---------------------------------------------------+


Update ~/.ssh/config on client as below:

#Client ~/.ssh/config
#---------------------------------------------------
Host ft-screening-underground-wave.trycloudflare.com
	Hostname %h
	User root
	Port 65287
	LogLevel ERROR
	UserKnownHostsFile /dev/null
	ProxyCommand cloudflared access ssh --hostname %h
#---------------------------------------------------
Note: Windows client users on PS/cmd, provide full path to cloudflared.exe in ProxyCommand
        Also applies to linux users if PATH to cloudflared isn't added to $PATH
        Ex: Instead of 
            `ProxyCommand cloudflared access ssh --hostname %h`
        use `ProxyCommand <complete_path_to_cloud

In [ ]:
launch_codeserver(user, hostname)

#Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
source_path = "/content/drive/Othercomputers/My Laptop/NBRC_Sourav_Banerjee_Dissertation/RNA analysis"

#Function definitions

In [ ]:
def common_sumrank_sorted_target(path: str):
  """Given a path to a gene folder containing transcripts' chop chop target sequence result, returns a pandas dataframe
    with common target sequences across the transcripts sorted by sum of the ranks across all transcripts

    #todo, list top 2 targets nearest by a specified distance
  Parameters:
  -----------
    path : str
      path to the parent gene folder
  """
  root, dirs, files =  next(os.walk(path))
  dataframes = []
  for dir in sorted(dirs):
    file_path = os.path.join(os.path.join(root,dir), 'chop_chop.tsv')
    dataframes.append(pd.read_csv(file_path, sep='\t'))

  target_sequences = []
  for dataframe in dataframes:
      target_sequences.append(set(list(dataframe['Target sequence'])))
      target_sequences[0] = target_sequences[0].intersection(target_sequences[-1])

  sumrank_targets = dict()
  for target in target_sequences[0]:
    sumrank_targets[target] = []
    for dataframe in dataframes:
      sumrank_targets[target].append(min(list(dataframe[dataframe['Target sequence'] == target]['Rank'])))
    sumrank_targets[target].append(sum(sumrank_targets[target]))

  sumrank_sorted_targets = sorted(sumrank_targets.items(), key=lambda x:x[1][-1])

  column_names = []
  data = []
  for dir in dirs:
    column_names.append('Rank_'+dir)
    data.append([])
  column_names.append('Rank_Sum')
  column_names.append('Target sequence')
  data.append([])
  data.append([])

  for tup in sumrank_sorted_targets:
    data[-1].append(tup[0])
    data[-2].append(tup[1][-1])
    for i in range(len(tup[1][:-1])):
      data[i].append(tup[1][i])

  sorted_target_df = {}
  for i, column_name in enumerate(column_names):
    sorted_target_df[column_name] = data[i]
  sorted_target_df = pd.DataFrame(sorted_target_df)
  sorted_target_df.to_csv(os.path.join(path,'sorted_'+root.split('/')[-1]+'.tsv'), sep='\t')

  return sorted_target_df


def



def pickle_save(variable: object, path: str):
  """Save python object to binary file using pickle dump
  """
  f = open(path, 'wb')
  pickle.dump(variable, f)



def pickle_load(path: str):
  """
    load saved python object from pickle dump file
  """
  f=open(path, 'rb')
  pickle.load(f)

In [ ]:
lncRNAs = os.listdir(source_path)
lncRNAs

['1700020I14Rik',
 '2410006H16Rik',
 'Pantr1',
 'gencode_reference',
 'Arc',
 'Pvt1',
 'Actb']

# Actb

In [ ]:
file_name = rbs_file.split('.')
dataframe = pd.read_csv(os.path.join(source_path, gene_name, 'RBS', RBS_files[4]))
brain_df = None
try:
  brain_df = dataframe[dataframe[dataframe.columns[1]] == 'Brain']
except Exception as e:
  print('Brain cell type not found')

brain_df = brain_df.reset_index()

In [ ]:
gene_name = 'Actb'
RBS_files = os.listdir(os.path.join(source_path, gene_name, 'RBS'))
brain_rbs_merge = {'cell_type':[], 'protein':[], 'data_source':[], 'score':[], 'RBS_location_start':[], 'RBS_location_end':[], 'transcript_id':[]}
for rbs_file in RBS_files:
  file_name = rbs_file.split('.')
  dataframe = pd.read_csv(os.path.join(source_path, gene_name, 'RBS', rbs_file))
  brain_df = None
  try:
    brain_df = dataframe[dataframe[dataframe.columns[1]] == 'Brain']
  except Exception as e:
    print('Brain cell type not found')

  brain_df = brain_df.reset_index()

  for index, data in brain_df.iterrows():
    brain_rbs_merge['protein'].append(data[1])
    brain_rbs_merge['cell_type'].append(data[2])
    brain_rbs_merge['data_source'].append(rbs_file.split('.')[0])
    brain_rbs_merge['score'].append(data[4])
    print(data[3])
    brain_rbs_merge['RBS_location_start'].append(int(data[3].split('-')[0].split(':')[-1]))
    brain_rbs_merge['RBS_location_end'].append(int(data[3].split('-')[1].split(';')[0]))
    brain_rbs_merge['transcript_id'].append(data[7])

pd.DataFrame(brain_rbs_merge).to_csv(os.path.join(source_path, gene_name, 'RBS', 'merge_all.csv'))


In [ ]:
merge_data = pd.read_csv(os.path.join(source_path, gene_name, 'RBS', 'merge_all.csv'))
rbs_location_start = list(merge_data['RBS_location_start'])
rbs_location_end = list(merge_data['RBS_location_end'])
rbs_protein_name = list(merge_data['protein'])
bed_file = open(os.path.join(source_path, gene_name, 'chop_chop.bed'), 'r')
tsv_file = open(os.path.join(source_path, gene_name, 'chop_chop.tsv'), 'r')
bed_lines = bed_file.readlines()
tsv_lines = tsv_file.readlines()
flagged_tsv_header = tsv_lines[0].split('\t')
flagged_tsv_header.insert(1,'RBS_Flag')
flagged_tsv_header = '\t'.join(flagged_tsv_header)
flagged_bed_lines = []
flagged_tsv_lines = []
flagged_bed_lines.append(bed_lines[0])
flagged_bed_lines.append(bed_lines[1])
flagged_tsv_lines.append(flagged_tsv_header)
for i,line in enumerate(bed_lines[2:]):
  bed_split = line.split('\t')
  tsv_split = tsv_lines[1+i].split('\t')
  target_sequence_start = int(bed_split[1])
  target_sequence_end = int(bed_split[2])
  tsv_split.insert(1,'NA')
  for start, end, protein_name in zip(rbs_location_start, rbs_location_end, rbs_protein_name):
    if (start >= target_sequence_start and start <= target_sequence_end) or (end >= target_sequence_start and end <= target_sequence_end):
      bed_split[8] = '0,0,255\n'
      tsv_split[1] = protein_name
  flagged_bed_lines.append('\t'.join(bed_split))
  flagged_tsv_lines.append('\t'.join(tsv_split))


flagged_bed_file = open(os.path.join(source_path, gene_name, 'flagged_chop_chop.bed'), 'w')
flagged_tsv_file = open(os.path.join(source_path, gene_name, 'flagged_chop_chop.tsv'), 'w')
flagged_bed_file.writelines(flagged_bed_lines)
flagged_tsv_file.writelines(flagged_tsv_lines)
flagged_bed_file.close()
flagged_tsv_file.close()

In [ ]:
flagged_tsv_file = open(os.path.join(source_path, gene_name, 'flagged_chop_chop.tsv'), 'r')
lines = flagged_tsv_file.readlines()
lines[0]

'Rank\tTarget sequence\tGenomic location\tGene\tIsoform\tGC content (%)\tSelf-complementarity\tLocal structure\tMM0\tMM1\tMM2\tMM3\tConstitutive\tIsoformsMM0\tIsoformsMM1\tIsoformsMM2\tIsoformsMM3\n'

In [ ]:
pd.read_csv(os.path.join(source_path, gene_name, 'chop_chop.tsv'), sep='\t')

,Rank,Target sequence,Genomic location,Gene,Isoform,GC content (%),Self-complementarity,Local structure,MM0,MM1,MM2,MM3,Constitutive,IsoformsMM0,IsoformsMM1,IsoformsMM2,IsoformsMM3
0,1,ACTTGACAACATTATTTATTTTTCTCTA,chr5:142904786,Actb,union,22,0,4.256879,0,0,0,0,0,ENSMUST00000165629,NaN,NaN,NaN
1,2,TATAAAACCCGGCGGCGCAACGCGCAGC,chr5:142906727,Actb,union,67,1,4.852630,0,0,0,0,0,ENSMUST00000100497,NaN,NaN,NaN
2,3,CAGCTTCTCAGCCACGCCCTTTCTCAAT,chr5:142905178,Actb,union,52,0,7.813893,0,0,0,0,0,"ENSMUST00000165629,ENSMUST00000164765",NaN,NaN,NaN
3,4,TGTCTTTCTTCTGCCGTTCTCCCATAGG,chr5:142905150,Actb,union,52,0,8.362614,0,0,0,0,0,"ENSMUST00000165629,ENSMUST00000164765",NaN,NaN,NaN
4,5,TTTGTCCCCTGAGCTTGGGCGCGCGCCC,chr5:142905794,Actb,union,74,2,6.937553,0,0,0,0,0,ENSMUST00000171419,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,88,GTACCCAGGCATTGCTGACAGGATGCAG,chr5:142904109,Actb,union,56,1,16.160929,14,6,6,7,0,"ENSMUST00000100497,ENSMUST00000167721",NaN,NaN,NaN
88,89,CTCACTGTCCACCTTCCAGCAGATGTGG,chr5:142903858,Actb,union,56,1,28.622429,16,5,4,8,0,"ENSMUST00000100497,ENSMUST00000167721",NaN,NaN,NaN
89,90,TACATTCAATTCCATCATGAAGTGTGAC,chr5:142904193,Actb,union,37,0,22.752786,2,21,2,22,0,"ENSMUST00000100497,ENSMUST00000167721",NaN,NaN,NaN
90,91,CTGGCACCACACCTTCTACAATGAGCTG,chr5:142905318,Actb,union,52,0,22.640107,16,11,8,3,0,"ENSMUST00000165629,ENSMUST00000164765,ENSMUST0...",NaN,NaN,NaN


In [ ]:
pd.read_csv(os.path.join(source_path, gene_name, 'flagged_chop_chop.tsv'), sep='\t')

,Rank,RBS_Flag,Target sequence,Genomic location,Gene,Isoform,GC content (%),Self-complementarity,Local structure,MM0,MM1,MM2,MM3,Constitutive,IsoformsMM0,IsoformsMM1,IsoformsMM2,IsoformsMM3
0,1,NaN,ACTTGACAACATTATTTATTTTTCTCTA,chr5:142904786,Actb,union,22,0,4.256879,0,0,0,0,0,ENSMUST00000165629,NaN,NaN,NaN
1,2,NaN,TATAAAACCCGGCGGCGCAACGCGCAGC,chr5:142906727,Actb,union,67,1,4.852630,0,0,0,0,0,ENSMUST00000100497,NaN,NaN,NaN
2,3,NaN,CAGCTTCTCAGCCACGCCCTTTCTCAAT,chr5:142905178,Actb,union,52,0,7.813893,0,0,0,0,0,"ENSMUST00000165629,ENSMUST00000164765",NaN,NaN,NaN
3,4,NaN,TGTCTTTCTTCTGCCGTTCTCCCATAGG,chr5:142905150,Actb,union,52,0,8.362614,0,0,0,0,0,"ENSMUST00000165629,ENSMUST00000164765",NaN,NaN,NaN
4,5,NaN,TTTGTCCCCTGAGCTTGGGCGCGCGCCC,chr5:142905794,Actb,union,74,2,6.937553,0,0,0,0,0,ENSMUST00000171419,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,88,U2AF2,GTACCCAGGCATTGCTGACAGGATGCAG,chr5:142904109,Actb,union,56,1,16.160929,14,6,6,7,0,"ENSMUST00000100497,ENSMUST00000167721",NaN,NaN,NaN
88,89,NaN,CTCACTGTCCACCTTCCAGCAGATGTGG,chr5:142903858,Actb,union,56,1,28.622429,16,5,4,8,0,"ENSMUST00000100497,ENSMUST00000167721",NaN,NaN,NaN
89,90,U2AF2,TACATTCAATTCCATCATGAAGTGTGAC,chr5:142904193,Actb,union,37,0,22.752786,2,21,2,22,0,"ENSMUST00000100497,ENSMUST00000167721",NaN,NaN,NaN
90,91,NaN,CTGGCACCACACCTTCTACAATGAGCTG,chr5:142905318,Actb,union,52,0,22.640107,16,11,8,3,0,"ENSMUST00000165629,ENSMUST00000164765,ENSMUST0...",NaN,NaN,NaN


#2410006H16Rik

In [ ]:
gene_name = '2410006H16Rik'
RBS_files = os.listdir(os.path.join(source_path, gene_name, 'RBS'))
brain_rbs_merge = {'cell_type':[], 'protein':[], 'data_source':[], 'score':[], 'RBS_location_start':[], 'RBS_location_end':[], 'transcript_id':[]}
for rbs_file in RBS_files:
  file_name = rbs_file.split('.')
  dataframe = pd.read_csv(os.path.join(source_path, gene_name, 'RBS', rbs_file))
  brain_df = None
  try:
    brain_df = dataframe[dataframe[dataframe.columns[1]] == 'Brain']
  except Exception as e:
    print('Brain cell type not found')

  brain_df = brain_df.reset_index()

  for index, data in brain_df.iterrows():
    brain_rbs_merge['protein'].append(data[1])
    brain_rbs_merge['cell_type'].append(data[2])
    brain_rbs_merge['data_source'].append(rbs_file.split('.')[0])
    brain_rbs_merge['score'].append(data[4])
    print(data[3])
    brain_rbs_merge['RBS_location_start'].append(int(data[3].split('-')[0].split(':')[-1]))
    brain_rbs_merge['RBS_location_end'].append(int(data[3].split('-')[1].split(';')[0]))
    brain_rbs_merge['transcript_id'].append(data[7])

pd.DataFrame(brain_rbs_merge).to_csv(os.path.join(source_path, gene_name, 'RBS', 'merge_all.csv'))


In [ ]:
merge_data = pd.read_csv(os.path.join(source_path, gene_name, 'RBS', 'merge_all.csv'))
rbs_location_start = list(merge_data['RBS_location_start'])
rbs_location_end = list(merge_data['RBS_location_end'])
rbs_protein_name = list(merge_data['protein'])
bed_file = open(os.path.join(source_path, gene_name,'ENSMUST00000131787', 'chop_chop.bed'), 'r')
tsv_file = open(os.path.join(source_path, gene_name,'ENSMUST00000131787', 'chop_chop.tsv'), 'r')
bed_lines = bed_file.readlines()
tsv_lines = tsv_file.readlines()
flagged_tsv_header = tsv_lines[0].split('\t')
flagged_tsv_header.insert(1,'RBS_Flag')
flagged_tsv_header = '\t'.join(flagged_tsv_header)
flagged_bed_lines = []
flagged_tsv_lines = []
flagged_bed_lines.append(bed_lines[0])
flagged_bed_lines.append(bed_lines[1])
flagged_tsv_lines.append(flagged_tsv_header)
for i,line in enumerate(bed_lines[2:]):
  bed_split = line.split('\t')
  tsv_split = tsv_lines[1+i].split('\t')
  target_sequence_start = int(bed_split[1])
  target_sequence_end = int(bed_split[2])
  tsv_split.insert(1,'NA')
  for start, end, protein_name in zip(rbs_location_start, rbs_location_end, rbs_protein_name):
    if (start >= target_sequence_start and start <= target_sequence_end) or (end >= target_sequence_start and end <= target_sequence_end):
      bed_split[8] = '0,0,255\n'
      tsv_split[1] = protein_name
  flagged_bed_lines.append('\t'.join(bed_split))
  flagged_tsv_lines.append('\t'.join(tsv_split))


flagged_bed_file = open(os.path.join(source_path, gene_name,'ENSMUST00000131787', 'flagged_chop_chop.bed'), 'w')
flagged_tsv_file = open(os.path.join(source_path, gene_name,'ENSMUST00000131787', 'flagged_chop_chop.tsv'), 'w')
flagged_bed_file.writelines(flagged_bed_lines)
flagged_tsv_file.writelines(flagged_tsv_lines)
flagged_bed_file.close()
flagged_tsv_file.close()

In [ ]:
brain_df

,RBPELAVL1FMR1LIN28AMBNL1MBNL2MSI2PABPC1,Cell & tissue typeBrainIntestinal_epitheliaMELMyoblastQuadricepsmESC,RBS location,"Score<span class=""glyphicon glyphicon-question-sign"" title=""Piranha score: Peak heights from the CLIP-seq data. PARalyzer score: T-to-C transition ratio, while higher ratio means more possible to bind RBPs. MiClip score: Probability of RBP binding. CIMS (CTK) score: Mismatch (HITS-CLIP) / Truncated (iCLIP) peak heights. PureCLIP score: Sum of log posterior probability ratio scores. ENCODE eCLIP score: -log10 P-value. "">",PhastCons score,PhyloP score,Transcript ID,Data accession,Overexpression Info
0,ELAVL1,Brain,chr5:142903610-142903631;-,13.0,1.000,4.914,ENSMUST00000100497,"GSE45148,GSM1098059",No
1,ELAVL1,Brain,chr5:142903610-142903631;-,13.0,1.000,4.914,ENSMUST00000167721,"GSE45148,GSM1098059",No
2,ELAVL1,Brain,chr5:142903611-142903632;-,13.0,1.000,4.849,ENSMUST00000100497,"GSE45148,GSM1098059",No
3,ELAVL1,Brain,chr5:142903611-142903632;-,13.0,1.000,4.849,ENSMUST00000167721,"GSE45148,GSM1098059",No
9,ELAVL1,Brain,chr5:142903598-142903619;-,6.0,1.000,5.156,ENSMUST00000100497,"GSE45148,GSM1098058",No
10,ELAVL1,Brain,chr5:142903598-142903619;-,6.0,1.000,5.156,ENSMUST00000167721,"GSE45148,GSM1098058",No
16,ELAVL1,Brain,chr5:142903610-142903631;-,5.0,1.000,4.914,ENSMUST00000100497,"GSE45148,GSM1098058",No
17,ELAVL1,Brain,chr5:142903610-142903631;-,5.0,1.000,4.914,ENSMUST00000167721,"GSE45148,GSM1098058",No
18,ELAVL1,Brain,chr5:142903611-142903632;-,5.0,1.000,4.849,ENSMUST00000100497,"GSE45148,GSM1098058",No
19,ELAVL1,Brain,chr5:142903611-142903632;-,5.0,1.000,4.849,ENSMUST00000167721,"GSE45148,GSM1098058",No


In [ ]:
common_sumrank_sorted_target(os.path.join(source_path, '1700020I14Rik'))

,Rank_ENSMUST00000153581,Rank_ENSMUST00000147425,Rank_Sum,Target sequence
0,1,1,2,GGAGCCTTTCGCGCCTCCGCCCCTGGGT
1,4,2,6,GTATAAATAAGTTCCTTCAACCCAAAAT
2,5,3,8,TATAAATAAGTTCCTTCAACCCAAAATG
3,6,4,10,ATAAATAAGTTCCTTCAACCCAAAATGG
4,7,5,12,CCAAAATGGGTACTTGGACAGATTTTTT
...,...,...,...,...
3502,3530,8207,11737,GTGTGTGTGTGTGTGTGTGTGTGTGTGG
3503,3528,8210,11738,TTGGTGTGTGTGTGTGTGTGTGTGTGTG
3504,3531,8212,11743,GGTGTGTGTGTGTGTGTGTGTGTGTGTG
3505,3532,8213,11745,GTGTGTGTGTGTGTGTGTGTGTGTGTGT


In [ ]:
df = pd.read_csv(os.path.join(source_path, lncRNAs[1], 'ENSMUST00000131787', 'chop_chop.tsv'), sep='\t')
df[df['Target sequence'] == 'CCGTGGCCTGCTGGTGACGGTCTGGAGC']

,Rank,Target sequence,Genomic location,Gene,Isoform,GC content (%),Self-complementarity,Local structure,MM0,MM1,MM2,MM3,Constitutive,IsoformsMM0,IsoformsMM1,IsoformsMM2,IsoformsMM3
1,2,CCGTGGCCTGCTGGTGACGGTCTGGAGC,ENSMUST00000131787.3:108,None,None,70,0,0,1,0,0,0,1,NaN,NaN,NaN,NaN


In [ ]:
common_sumrank_sorted_target(os.path.join(source_path, '2410006H16Rik'))

,Rank_ENSMUST00000131787,Rank_Sum,Target sequence
0,1,1,AGGAGTGGTCTATTCCTAGCCCTTAAGA
1,2,2,CCGTGGCCTGCTGGTGACGGTCTGGAGC
2,3,3,CGTGGCCTGCTGGTGACGGTCTGGAGCG
3,4,4,GTGGCCTGCTGGTGACGGTCTGGAGCGA
4,5,5,TGGCCTGCTGGTGACGGTCTGGAGCGAT
...,...,...,...
574,575,575,AATCCTGTCTGTACCACAGGATCTTCTA
575,576,576,ATTTGGTAATCCTGTCTGTACCACAGGA
576,577,577,TGGTAATCCTGTCTGTACCACAGGATCT
577,578,578,TTTGGTAATCCTGTCTGTACCACAGGAT


In [ ]:
df = pd.read_csv(os.path.join(source_path, lncRNAs[2], 'sorted_Pantr1.tsv'), sep='\t')
df[df['Target sequence'] == 'CCGTGGCCTGCTGGTGACGGTCTGGAGC']

In [ ]:
common_sumrank_sorted_target(os.path.join(source_path, 'Pantr1'))

,Rank_ENSMUST00000078844,Rank_ENSMUST00000176052,Rank_ENSMUST00000180689,Rank_ENSMUST00000181593,Rank_ENSMUST00000181725,Rank_ENSMUST00000188153,Rank_Sum,Target sequence
0,23,52,17,51,4,11,158,TTATGAATAAAGTTCTATAGGACACAGA
1,24,53,19,52,5,13,166,TATGAATAAAGTTCTATAGGACACAGAC
2,92,131,587,74,32,584,1500,CTTATGAATAAAGTTCTATAGGACACAG
3,159,153,735,288,195,686,2216,TCTTATGAATAAAGTTCTATAGGACACA
4,168,169,820,308,216,742,2423,GGAACTCAGTGAATATTGCCCTGGGTCC
...,...,...,...,...,...,...,...,...
365,533,525,1113,674,580,1063,4488,GACATTCTTTGTTACAATGAACCTAGAA
366,534,526,1114,675,581,1064,4494,ACATTCTTTGTTACAATGAACCTAGAAG
367,535,527,1115,676,582,1065,4500,CATTCTTTGTTACAATGAACCTAGAAGA
368,536,528,1116,677,583,1066,4506,ATTCTTTGTTACAATGAACCTAGAAGAA


#Oligo preparation

In [ ]:
from Bio.Seq import Seq

Seq('GAAGGGGACTAAAACGGCAACGAAGGAGCTGCAAAGAAGCTGTGATTTAGACTACCCC').reverse_complement()

Seq('GGGGTAGTCTAAATCACAGCTTCTTTGCAGCTCCTTCGTTGCCGTTTTAGTCCCCTTC')